In [94]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [95]:
# Create a 30x30 rect lattice using a dictionary
# then reduce each 10x10 plot into AF to get a 3x3 rect lattice
lattice = {(x,y): False for x in range(30) for y in range(30)}

In [96]:
from collections import defaultdict

def reduce_lattice(lattice, reduced_size=3, plot_size=10):
    _lattice = defaultdict(int)
    for coords, cell in lattice.items():
        _coords = (divmod(coords[0], plot_size)[0], divmod(coords[1], plot_size)[0])
        _lattice[_coords] += cell
    return {_coords: cell_sum / float(plot_size**2) 
        for _coords, cell_sum in _lattice.items()}

In [97]:
lattice[(0,0)] = True
rlattice = reduce_lattice(lattice)
# Check if reduce_lattice works
rlattice

{(0, 0): 0.01,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (1, 0): 0.0,
 (1, 1): 0.0,
 (1, 2): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.0}

In [98]:
def check_validity(coords:tuple, x_lim:tuple, y_lim:tuple) -> bool:
    if coords[0] < x_lim[0]: return False
    if coords[0] > x_lim[1]: return False
    if coords[1] < y_lim[0]: return False
    if coords[1] > y_lim[1]: return False
    return True   

In [151]:
# Run a toy simulation
# No new mutation
# No back mutation
# No spontaneous death
# Only competition between mutant and wildtype changes the counts
x_range = 100
y_range = 100
plot_size = 10
mutant_start_coords = (50, 50)  # in (1,1) in 3x3 reduced lattice
reduced_size = 10

lattice = {(x,y): False for x in range(x_range) for y in range(y_range)}

lattice[mutant_start_coords] = True
replace_by_mutant_prob = 0.51

mut_set = set([mutant_start_coords,])
wt_set = set([k for k in lattice.keys()])
wt_set.discard(mutant_start_coords)
c = 0
rlattice = reduce_lattice(lattice, reduced_size=reduced_size, plot_size=plot_size)
print(f'[{c}]')
for i in range(reduced_size):
    row = [f'{rlattice[(j, i)]:.2f}' for j in range(reduced_size)]
    print(' '.join(row))

# print(f'{rlattice[(0, 2)]:.2f} {rlattice[(1, 2)]:.2f} {rlattice[(2, 2)]:.2f}')
# print(f'{rlattice[(0, 1)]:.2f} {rlattice[(1, 1)]:.2f} {rlattice[(2, 1)]:.2f}')
# print(f'{rlattice[(0, 0)]:.2f} {rlattice[(1, 0)]:.2f} {rlattice[(2, 0)]:.2f}')

# print('')
# print(f'{lattice[(0, 5)]:.0f} {lattice[(1, 5)]:.0f} {lattice[(2, 5)]:.0f} {lattice[(3, 5)]:.0f} {lattice[(4, 5)]:.0f} {lattice[(5, 5)]:.0f}')
# print(f'{lattice[(0, 4)]:.0f} {lattice[(1, 4)]:.0f} {lattice[(2, 4)]:.0f} {lattice[(3, 4)]:.0f} {lattice[(4, 4)]:.0f} {lattice[(5, 4)]:.0f}')
# print(f'{lattice[(0, 3)]:.0f} {lattice[(1, 3)]:.0f} {lattice[(2, 3)]:.0f} {lattice[(3, 3)]:.0f} {lattice[(4, 3)]:.0f} {lattice[(5, 3)]:.0f}')
# print(f'{lattice[(0, 2)]:.0f} {lattice[(1, 2)]:.0f} {lattice[(2, 2)]:.0f} {lattice[(3, 2)]:.0f} {lattice[(4, 2)]:.0f} {lattice[(5, 2)]:.0f}')
# print(f'{lattice[(0, 1)]:.0f} {lattice[(1, 1)]:.0f} {lattice[(2, 1)]:.0f} {lattice[(3, 1)]:.0f} {lattice[(4, 1)]:.0f} {lattice[(5, 1)]:.0f}')
# print(f'{lattice[(0, 0)]:.0f} {lattice[(1, 0)]:.0f} {lattice[(2, 0)]:.0f} {lattice[(3, 0)]:.0f} {lattice[(4, 0)]:.0f} {lattice[(5, 0)]:.0f}')
# print('')


c = 1
while not (len(wt_set) == 0 or len(mut_set) == 0):
    for mut_coords in mut_set:
        # get neighbors
        nbr_coords_list = [(mut_coords[0]+i, mut_coords[1]+j) for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0)]
        # iterate valid neighbor coords 
        # check if cell competition is necessary
        nbr_coords_list = list(filter(lambda x: check_validity(x, (0,x_range-1), (0,y_range-1)), nbr_coords_list))
        [coords
            for coords in filter(lambda x: check_validity(x, (0,x_range-1), (0,y_range-1)), nbr_coords_list)
            if not lattice[coords]]
        # print(mut_coords, nbr_coords_list)
        if len(nbr_coords_list) > 0:
            np.random.shuffle(nbr_coords_list)
            nbr_coords = nbr_coords_list[0]
            # If chosen neighbor is already a mutant, ignore and continue to next
            if lattice[nbr_coords]:
                continue
            # Cell competition occurs between mutant and wt neighbor
            if np.random.random() < replace_by_mutant_prob:
                lattice[nbr_coords] = True
            else:
                lattice[mut_coords] = False

    # update mut_set and wt_set
    mut_set = set()
    wt_set = set()
    for k, v in lattice.items():
        if v:
            mut_set.add(k)
        else:
            wt_set.add(k)
    
    # print(i, reduce_lattice(lattice))
    if c % 100 == 0:
        rlattice = reduce_lattice(lattice, reduced_size=reduced_size, plot_size=plot_size)
        print(f'[{c}]')
        for i in range(reduced_size):
            row = [f'{rlattice[(j, i)]:.2f}' for j in range(reduced_size)]
            print(' '.join(row))
        # print(f'{rlattice[(0, 2)]:.2f} {rlattice[(1, 2)]:.2f} {rlattice[(2, 2)]:.2f}')
        # print(f'{rlattice[(0, 1)]:.2f} {rlattice[(1, 1)]:.2f} {rlattice[(2, 1)]:.2f}')
        # print(f'{rlattice[(0, 0)]:.2f} {rlattice[(1, 0)]:.2f} {rlattice[(2, 0)]:.2f}')

    # print('')
    # print(f'{lattice[(0, 5)]:.0f} {lattice[(1, 5)]:.0f} {lattice[(2, 5)]:.0f} {lattice[(3, 5)]:.0f} {lattice[(4, 5)]:.0f} {lattice[(5, 5)]:.0f}')
    # print(f'{lattice[(0, 4)]:.0f} {lattice[(1, 4)]:.0f} {lattice[(2, 4)]:.0f} {lattice[(3, 4)]:.0f} {lattice[(4, 4)]:.0f} {lattice[(5, 4)]:.0f}')
    # print(f'{lattice[(0, 3)]:.0f} {lattice[(1, 3)]:.0f} {lattice[(2, 3)]:.0f} {lattice[(3, 3)]:.0f} {lattice[(4, 3)]:.0f} {lattice[(5, 3)]:.0f}')
    # print(f'{lattice[(0, 2)]:.0f} {lattice[(1, 2)]:.0f} {lattice[(2, 2)]:.0f} {lattice[(3, 2)]:.0f} {lattice[(4, 2)]:.0f} {lattice[(5, 2)]:.0f}')
    # print(f'{lattice[(0, 1)]:.0f} {lattice[(1, 1)]:.0f} {lattice[(2, 1)]:.0f} {lattice[(3, 1)]:.0f} {lattice[(4, 1)]:.0f} {lattice[(5, 1)]:.0f}')
    # print(f'{lattice[(0, 0)]:.0f} {lattice[(1, 0)]:.0f} {lattice[(2, 0)]:.0f} {lattice[(3, 0)]:.0f} {lattice[(4, 0)]:.0f} {lattice[(5, 0)]:.0f}')
    # print('')

    c += 1

# Final
rlattice = reduce_lattice(lattice, reduced_size=reduced_size, plot_size=plot_size)
print(f'[{c}]')
for i in range(reduced_size):
    row = [f'{rlattice[(j, i)]:.2f}' for j in range(reduced_size)]
    print(' '.join(row))
# print(f'{rlattice[(0, 2)]:.2f} {rlattice[(1, 2)]:.2f} {rlattice[(2, 2)]:.2f}')
# print(f'{rlattice[(0, 1)]:.2f} {rlattice[(1, 1)]:.2f} {rlattice[(2, 1)]:.2f}')
# print(f'{rlattice[(0, 0)]:.2f} {rlattice[(1, 0)]:.2f} {rlattice[(2, 0)]:.2f}')

[0]
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[100]
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.02 0.01 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.06 0.17 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 

KeyboardInterrupt: 